In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from utils import *

In [10]:
path = "data/car evaluation.csv"
data = pd.read_csv(path, header=None)
data_1 = data[data.iloc[:, 6] == 1]
data_1 = data_1.sample(n=400, random_state=1234)
data_2 = data[data.iloc[:, 6] == 2]
data_all = pd.concat([data_1, data_2], axis=0)
data_all = data_all.sample(frac=1, random_state=1234).reset_index(drop=True)

feature_names = [
    "buying_price",
    "maintanace_price",
    "number_of_doors",
    "capacity",
    "boot_size",
    "safety"
]
class_names = ["unaccetable", "good"]

target_map = {0: 0, 1: 0, 2: 1, 3: 1, 4: 1}
num_criteria = 6
data_input = data_all.iloc[:, :num_criteria]
data_target = data_all[num_criteria].apply(lambda x: target_map[x])

data_input = data_input.values.reshape(-1, 1, num_criteria)

X_train, X_test, y_train, y_test = train_test_split(
    data_input, data_target.values, test_size=0.2, random_state=1234
)


train_dataloader = CreateDataLoader(X_train, y_train)
test_dataloader = CreateDataLoader(X_test, y_test)

In [3]:
feature_names = [
    "buying_price",
    "maintanace_price",
    "number_of_doors",
    "capacity",
    "boot_size",
    "safety"
]
class_names = ["unaccetable", "good"]

In [21]:
import torch.nn as nn

class CarEvaluationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CarEvaluationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Define model parameters
input_size = num_criteria  # Number of features
hidden_size = 16  # Number of neurons in the hidden layer
output_size = 1 # Number of output classes

# Instantiate the model
model = CarEvaluationModel(input_size, hidden_size, output_size)
print(model)

CarEvaluationModel(
  (fc1): Linear(in_features=6, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [42]:
model.eval()
with torch.no_grad():
    X_traintt = torch.tensor(X_train, dtype=torch.float32)
    y_traintt = torch.tensor(y_train, dtype=torch.float32)
    # Forward pass
    y_traintt = y_traintt.view(-1, y_traintt.shape[0])
    print(y_traintt.shape)
    print(X_traintt.shape)
    ou = model(X_traintt)
    # torch reshape
    ou = ou.view(-1,ou.shape[0])
    print(ou.shape)

torch.Size([1, 627])
torch.Size([627, 1, 6])
torch.Size([1, 627])


In [61]:
from sklearn.metrics import accuracy_score, roc_auc_score

# train the model
def train_model(model, train_dataloader, criterion, optimizer, num_epochs=300):
    model.train()
    for epoch in range(num_epochs):
        all_labels = []
        all_preds = []
        for inputs, labels in train_dataloader:
            inputs = inputs.float()
            labels = labels.float()
            labels = labels.view(-1, labels.shape[0])

            # Forward pass
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.shape[0])
            
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Collect predictions and labels for metrics
            _, preds = torch.max(outputs, 1)
            print(preds.shape)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels = np.array(all_labels)
            all_preds = np.array(all_preds)

            print(all_labels.shape)
            print(all_preds.shape)


        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        # auc_roc = roc_auc_score(all_labels, all_preds)

        # if (epoch + 1) % 10 == 0:
        #     print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}, AUC-ROC: {auc_roc:.4f}')

In [62]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Train the model
train_model(model, train_dataloader, criterion, optimizer, num_epochs=300)
# Evaluate the model
# def evaluate_model(model, test_dataloader):
#     model.eval()
#     all_labels = []
#     all_preds = []
#     with torch.no_grad():
#         for inputs, labels in test_dataloader:
#             inputs = inputs.float()
#             labels = labels.long()

#             # Forward pass
#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)

#             # Collect predictions and labels for metrics
#             all_labels.extend(labels.cpu().numpy())
#             all_preds.extend(preds.cpu().numpy())

#     # accuracy = accuracy_score(all_labels, all_preds)
#     # auc_roc = roc_auc_score(all_labels, all_preds)
#     # print(f'Test Accuracy: {accuracy:.4f}, Test AUC-ROC: {auc_roc:.4f}')
# # Evaluate the model
# evaluate_model(model, test_dataloader)


torch.Size([1])
(1, 627)
(1,)


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets